In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import os
import csv

# for progress bars
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

# API Key
from config import api_key

#ZillowWrapper
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults

0it [00:00, ?it/s]


In [43]:
#Read csv
address_zip_df = pd.read_csv('ChicagoAddress.csv').dropna()
address_zip_df.head()


,Address,Zip
0,5 14th,60605
1,8 9th,60605
2,234 Ashland,60605
3,1733 Congress,60605
4,1753 Congress,60605


In [44]:
test_data = address_zip_df.head(20)

In [45]:
test_data.shape

(20, 2)

In [46]:
address_zip_df.shape

(18293, 2)

In [47]:
address = '4244 N Pulaski'
zipcode = '60641'

zillow_data = ZillowWrapper(api_key)
response = zillow_data.get_deep_search_results(address, zipcode)
result = GetDeepSearchResults(response)
result.home_size

In [48]:
bad_address = "234 Ashland"
bad_zip = "60605"

In [49]:
good_address = "125 E 13th St 1001"
good_zip = "60605"

In [50]:
zillow_data = ZillowWrapper(api_key)

def get_zillow_info(address,zipcode):
    try:
        response = zillow_data.get_deep_search_results(address, zipcode)
        result = GetDeepSearchResults(response)
        
        # get zollow_id value
        try:
            zillow_id = result.zillow_id
        except AttributeError: 
            zillow_id = None
        
        # 1get tax value
        try:
            tax = result.tax_value
        except AttributeError: 
            tax = None
        # 2get latitude
        try:
            lat = result.latitude
        except AttributeError:
            lat = None
        # 3home value
        try:    
            hv = result.home_value
        except AttributeError:
            hv = None
        
        # 4home type 
        try:    
            home_type = result.home_type
        except AttributeError:
            home_type = None
            
        # 5home size
        try:    
            home_size = result.home_size
        except AttributeError:
            home_size = None
        
        # 6bathrooms
        try:    
            bathrooms = result.bathrooms
        except AttributeError:
            bathrooms = None
        
        # 7bedrooms
        try:    
            bedrooms = result.bedrooms
        except AttributeError:
            bedrooms = None
            
        # 8last sold date
        try:    
            last_sold_date = result.last_sold_date
        except AttributeError:
            last_sold_date = None
        
        # 9last sold price
        try:    
            last_sold_price = result.last_sold_price
        except AttributeError:
            last_sold_price = None
            
       # 10zestimate amount
        try:    
            zestimate_amount = result.zestimate_amount
        except AttributeError:
            zestimate_amount = None
        # 11zestimate last updated
        try:    
            zestimate_last_updated = result.zestimate_last_updated
        except AttributeError:
            home_type = None
        # 12zestimate value change
        try:    
            zestimate_value_change = result.zestimate_value_change
        except AttributeError:
            zestimate_value_change = None
        
        final = [zillow_id,tax,lat,hv,home_type,
                 home_size,bathrooms,bedrooms,
                 last_sold_date,last_sold_price,zestimate_amount,
                 zestimate_last_updated,zestimate_value_change]
        
    except:
        final = []
    return final

In [51]:
get_zillow_info(bad_address,bad_zip)

[]

In [52]:
test_data["zillow_info"] = test_data.progress_apply(lambda row: get_zillow_info(row["Address"],row["Zip"]),axis=1)

20it [00:14,  6.79it/s]
/anaconda2/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
test_data.head()#["zillow_info"][10][10]

,Address,Zip,zillow_info
0,5 14th,60605,"[60204228, 248760.0, 41.8682, None, Condominiu..."
1,8 9th,60605,"[119088436, 40600.0, 41.8681, None, Miscellane..."
2,234 Ashland,60605,[]
3,1733 Congress,60605,[]
4,1753 Congress,60605,[]


In [54]:
# turn the zillow_info column which is a list into dataframe
zillow_info_df = pd.DataFrame(test_data.zillow_info.values.tolist())

In [55]:
zillow_info_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,60204228,248760.0,41.8682,None,Condominium,1250,2.0,2,05/27/2004,295000,393921,04/02/2019,49273
1,119088436,40600.0,41.8681,None,Miscellaneous,None,None,0,03/09/2018,300000,None,01/01/1970,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None


In [56]:
# CHANGE COLUMN NAME
zillow_info_df = zillow_info_df.rename(columns={0:"Zillow ID", 1:"Tax"})

#df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})
zillow_info_df

,Zillow ID,Tax,2,3,4,5,6,7,8,9,10,11,12
0,60204228,248760.0,41.8682,None,Condominium,1250,2.0,2,05/27/2004,295000,393921,04/02/2019,49273
1,119088436,40600.0,41.8681,None,Miscellaneous,None,None,0,03/09/2018,300000,None,01/01/1970,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None
5,2093316676,None,41.885242,None,Apartment,685,1.0,1,02/01/2017,2415,None,01/01/1970,None
6,2095670559,None,41.884006,None,Apartment,785,1.0,1,11/10/2016,2500,None,01/01/1970,None
7,None,None,None,None,None,None,None,None,None,None,None,None,None
8,70474983,513000.0,41.86567,None,Condominium,1615,2.0,2,07/20/2012,525000,657729,04/02/2019,-326
9,70474813,448300.0,41.865577,None,Condominium,1716,3.0,3,12/07/2011,275000,656607,04/02/2019,1598


In [57]:
# combine zillow info dataframe with address info, to make sure they line up
final_zillow = pd.concat([test_data,
                          zillow_info_df],axis=1)

In [58]:
# check results
final_zillow.head()

,Address,Zip,zillow_info,Zillow ID,Tax,2,3,4,5,6,7,8,9,10,11,12
0,5 14th,60605,"[60204228, 248760.0, 41.8682, None, Condominiu...",60204228,248760.0,41.8682,None,Condominium,1250,2.0,2,05/27/2004,295000,393921,04/02/2019,49273
1,8 9th,60605,"[119088436, 40600.0, 41.8681, None, Miscellane...",119088436,40600.0,41.8681,None,Miscellaneous,None,None,0,03/09/2018,300000,None,01/01/1970,None
2,234 Ashland,60605,[],None,None,None,None,None,None,None,None,None,None,None,None,None
3,1733 Congress,60605,[],None,None,None,None,None,None,None,None,None,None,None,None,None
4,1753 Congress,60605,[],None,None,None,None,None,None,None,None,None,None,None,None,None


In [59]:
# check file path, just like `pwd` in terminal 
import os
os.getcwd()

'/Users/flowe/Desktop/Northwestern/NUCHI201902DATA3/NW-Bootcamp-Projects'

In [60]:
# drop zillow_info column AND save file
final_zillow.drop(columns=["zillow_info"]).to_csv("final_zillow_info.csv",index=None)